In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from textblob import TextBlob

In [ ]:
# Load the dataset
df = pd.read_csv("/workspaces/Sentiment_Analyzer/Sentiment_Analyzer/dataset/swahili.csv")

# Text preprocessing
stop_words = stopwords.words("english")
df["maneno"] = df["maneno"].apply(lambda x: " ".join([word for word in re.sub('[^a-zA-Z0-9\s]', '', x).split() if word not in stop_words]))

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df["maneno"], df["lugha"], test_size=0.3, random_state=42)


In [ ]:
# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Train an SVM classifier
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train_vec, y_train)

In [ ]:
# Predict on the test set
y_pred = svm.predict(X_test_vec)

In [ ]:
# Calculate accuracy score
acc = accuracy_score(y_test, y_pred)*100
print("Accuracy:", acc)

In [ ]:
# Load the new data from a CSV file
new_data = pd.read_csv("/workspaces/Sentiment_Analyzer/Sentiment_Analyzer/dataset/swahili_test.csv")

In [ ]:
# Preprocess the new data
new_data["maneno"] = new_data["maneno"].apply(lambda x: " ".join([word for word in re.sub('[^a-zA-Z0-9\s]', '', x).split() if word not in stop_words]))


In [ ]:
# Vectorize the new data
new_data_vec = vectorizer.transform(new_data["maneno"])

# Predict the sentiment of the new data
new_data["lugha"] = svm.predict(new_data_vec)

# Calculate the polarity scores of the test data and new data
test_data_polarity = [TextBlob(text).sentiment.polarity for text in X_test]
new_data_polarity = [TextBlob(text).sentiment.polarity for text in new_data["maneno"]]

In [ ]:
# Save the polarity scores to CSV files
#X_test_df = pd.DataFrame(X_test)
#test_data_polarity_df = pd.DataFrame(test_data_polarity, columns=["polarity"])
#test_data_polarity_df = pd.concat([X_test_df, test_data_polarity_df], axis=1)
#test_data_polarity_df.to_csv("Sentiment_Analyzer/dataset/test_data_polarity.csv", index=False)

#new_data_polarity_df = pd.DataFrame(new_data["maneno"], columns=["maneno"])
#new_data_polarity_df["polarity"] = new_data_polarity
#new_data_polarity_df.to_csv("Sentiment_Analyzer/dataset/new_data_polarity.csv", index=False)

In [ ]:
# Calculate the precision, recall, F1 score, and support for the test data and new data
precision_test, recall_test, f1_score_test, support_test = precision_recall_fscore_support(y_test, y_pred, average='weighted')
precision_new, recall_new, f1_score_new, support_new = precision_recall_fscore_support(new_data["lugha"], new_data["lugha"], average='weighted')


In [ ]:
# Calculate the accuracy for the test data and new data
accuracy_test = accuracy_score(y_test, y_pred)*100
accuracy_new = accuracy_score(new_data["lugha"], new_data["lugha"])*100


In [ ]:
# Print the accuracy, F1 score, precision, and recall for the test data and new data
print("Test data:")
print("Accuracy:", accuracy_test)
print("F1 score:", f1_score_test)
print("Precision:", precision_test)
print("Recall:", recall_test)

print("New data:")
print("Accuracy:", accuracy_new)
print("F1 score:", f1_score_new)
print("Precision:", precision_new)
print("Recall:", recall_new)